In [51]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from networkx.algorithms import bipartite
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

In [52]:
data = pd.read_csv('movielens-20m/rating.csv')
data = data.iloc[:10000]
movies_df = pd.read_csv("movielens-20m/movie.csv")
tags_df = pd.read_csv("movielens-20m/tag.csv")
genome_scores_raw = pd.read_csv("movielens-20m/genome_scores.csv")
# Filter genome_scores to only have movieIds present in the data dataframe
genome_scores = genome_scores_raw[genome_scores_raw['movieId'].isin(data['movieId'].unique())]

train_data = data.copy()
test_data = pd.DataFrame()
 
movie_num_user_rated_counts = train_data['movieId'].value_counts()
# Create a list of movie IDs with 50 or more ratings
popular_movies = movie_num_user_rated_counts[movie_num_user_rated_counts >= 10].index.tolist()
# Filter the DataFrame to keep only the rows with movies that meet the threshold
train_data = train_data[train_data['movieId'].isin(popular_movies)]

for user_id in train_data['userId'].unique():
    user_ratings = train_data[train_data['userId'] == user_id]
    
    if len(user_ratings) > 1:
        test_rating = user_ratings.sample()
        test_data = pd.concat([test_data, test_rating])
        train_data.drop(test_rating.index, inplace=True)

In [53]:
ratings_df = train_data
# Create a new graph
def makegraph():
    G = nx.Graph()

    # 2. Adding nodes and edges
    # Add user nodes
    for user_id in ratings_df['userId'].unique():
        G.add_node('u_'+str(user_id), bipartite=0)  # Add user node with a bipartite attribute of 0

    # Add movie nodes with title and genres as attributes
    for _, row in movies_df.iterrows():
        movie_id = 'm_' + str(row['movieId'])
        G.add_node(movie_id, bipartite=1, title=row['title'], genres=row['genres'].split('|'))

    # Add edges based on ratings with rating and timestamp as attributes
    for _, row in ratings_df.iterrows():
        user_id = 'u_'+str(row['userId'])
        movie_id = 'm_' + str(row['movieId'])
        G.add_edge(user_id, movie_id, rating=row['rating'], timestamp=row['timestamp'])

    # Add tag data as an attribute to the movie nodes
    for _, row in tags_df.iterrows():
        movie_id = 'm_' + str(row['movieId'])
        if 'tags' not in G.nodes[movie_id]:
            G.nodes[movie_id]['tags'] = []
        G.nodes[movie_id]['tags'].append({'tag': row['tag'], 'timestamp': row['timestamp'], 'userId': row['userId']})

    # Ensure the graph is bipartite
    assert bipartite.is_bipartite(G)
    return G
G = makegraph()

In [54]:
def predict_rating(G, user, movie):
    neighbors = list(G.neighbors(movie))
    if not neighbors:
        return np.mean([attr['rating'] for _, _, attr in G.edges(data=True) if 'rating' in attr])

    sim_weights = []
    user_ratings = []
    for neighbor in neighbors:
        # Jaccard similarity as an example, but can be changed
        common_movies = list(nx.common_neighbors(G, user, neighbor))
        sim = len(common_movies) / (G.degree(user) + G.degree(neighbor) - len(common_movies))
        rating = G[neighbor][movie]['rating']
        
        sim_weights.append(sim)
        user_ratings.append(rating)
    
    return np.dot(user_ratings, sim_weights) / sum(sim_weights)


In [108]:
def random_walk_legacy(G, start_node, alpha=0.85, walk_length=10):
    """Perform a random walk on graph G starting from node start_node."""
    current_node = start_node
    path = [current_node]
    
    for _ in range(walk_length):
        neighbors = list(G.neighbors(current_node))
        if random.random() < alpha and neighbors:
            current_node = random.choice(neighbors)
        else:
            current_node = start_node
        path.append(current_node)
    
    return path
def weighted_choice(neighbors, weights, heur='default',seed=None):
    if seed:
        random.seed(seed)
    if heur=='default':
        total_sum = sum(weights)
        normalized_list = [x / total_sum for x in weights]
        return random.choices(neighbors,weights)[0]
    else:
        total_sum = sum(weights)
        normalized_list = [x / total_sum for x in weights]
        return random.choices(neighbors,weights)[0]

def random_walk(G, start_node, alpha=0.85, walk_length=10, seed=None):
    """Perform a random walk on graph G starting from node start_node."""
    if seed:
        random.seed(seed)
    current_node = start_node
    path = [current_node]
    
    for _ in range(walk_length):
        seed=seed+1
        neighbors = list(G.neighbors(current_node))
        
        if neighbors:
            # Get weights (ratings) of the edges to the neighbors
            weights = [G[current_node][neighbor].get('rating', 1) for neighbor in neighbors]
            
            if random.random() < alpha:
                current_node = weighted_choice(neighbors, weights,seed=seed)
            else:
                current_node = start_node
        else:
            current_node = start_node
            
        path.append(current_node)
    
    return path
def personalized_pagerank_recommendations(G, user, alpha=0.85, num_walks=10000, walk_length=10, seed=None):
    """Generate recommendations and explanations using Personalized PageRank via random walks."""
    # Perform random walks and keep track of visit counts
    visit_counts = {node: 0 for node in G.nodes()}
    all_paths = []
    user = 'u_'+str(user)
    
    for _ in range(num_walks):
        seed=seed+10*walk_length
        path = random_walk(G, user, alpha, walk_length, seed)
        all_paths.append(path)
        for node in path:
            visit_counts[node] += 1
    
    # Normalize visit counts to get a probability distribution
    total_visits = sum(visit_counts.values())
    ppr = {node: count/total_visits for node, count in visit_counts.items()}
    
    # Filter for movies and sort by PPR score
    movies = [node for node in ppr.keys() if G.nodes[node]['bipartite'] == 1 and node not in G.neighbors(str(user))]
    sorted_movies = sorted(movies, key=lambda x: ppr[x], reverse=True)
    
    # Generate explanations for the top 10 movies
    explanations = {}
    significant_neighbors = {}
    contributing_paths_all = {}
    for movie in sorted_movies[:10]:
        # Find paths that contributed to the movie's score
        contributing_paths = [path for path in all_paths if movie in path]
        
        # Count the frequency of each neighbor leading to the movie
        neighbor_counts = {}
        for path in contributing_paths:
            for i in range(len(path) - 1):
                if path[i+1] == movie:
                    neighbor = path[1]
                    neighbor_counts[neighbor] = neighbor_counts.get(neighbor, 0) + 1

        # Identify the most significant neighbor
        sorted_neighbors = sorted(neighbor_counts, key=neighbor_counts.get, reverse=True)
        most_significant_neighbor = next((n for n in sorted_neighbors if not n.startswith('u_') and not n == movie), None)
        if most_significant_neighbor:
            significant_neighbors[movie] = most_significant_neighbor
        contributing_paths_all[movie] = contributing_paths
    return sorted_movies[:10], contributing_paths_all, significant_neighbors,visit_counts


In [104]:
def id_to_name(movie_id, movies_df):
    movie_row = movies_df[movies_df['movieId'] == movie_id]
    if not movie_row.empty:
        return movie_row['title'].iloc[0]
    else:
        return None

In [106]:
user = train_data['userId'].sample().iloc[0]
movies, paths, neigbors,visit_counts = personalized_pagerank_recommendations(G,user)
movie1 = int(next(iter(neigbors))[2:])
movie2 = int(neigbors.get(next(iter(neigbors)))[2:])
print(f"numbers:{movie1},{movie2}")
print(f"names:{id_to_name(movie1, movies_df)},{id_to_name(movie2, movies_df)}")

numbers:356,4022
names:Forrest Gump (1994),Cast Away (2000)


In [119]:
def compute_CF(G, user, alpha=0.85, num_walks=10000, walk_length=10,seed=None):
    
    movies, paths, _, visit_counts = personalized_pagerank_recommendations(G,user,alpha=alpha,num_walks=num_walks,walk_length=walk_length,seed=seed)
    path = next(iter(paths.items()))[1][0]
    for i in range(len(path) - 1):
        if G.has_edge(path[i], path[i+1]):
            G.remove_edge(path[i], path[i+1])
    movies_modified, _, _, visit_counts_modified = personalized_pagerank_recommendations(G,user,alpha=alpha,num_walks=num_walks,walk_length=walk_length,seed=seed)
    first_movie = movies[0]

    if first_movie in movies_modified:
        error = abs(visit_counts[first_movie] - visit_counts_modified[first_movie])
    else:
        error = visit_counts[first_movie]

    print(f"evaluation error ({first_movie}): {error} first_visits: {visit_counts[first_movie]}")

    return error

In [120]:
errors = []
for user in test_data['userId']:
    G_test = G.copy()
    errors.append(compute_CF(G_test,user,num_walks=1000,seed=42))

evaluation error (m_110): 1 first_visits: 35
evaluation error (m_356): 4 first_visits: 42
evaluation error (m_110): 5 first_visits: 38
evaluation error (m_457): 5 first_visits: 42
evaluation error (m_356): 13 first_visits: 34
evaluation error (m_356): 7 first_visits: 42
evaluation error (m_318): 4 first_visits: 41
evaluation error (m_593): 9 first_visits: 42
evaluation error (m_318): 7 first_visits: 44
evaluation error (m_318): 3 first_visits: 46
evaluation error (m_590): 1 first_visits: 33
evaluation error (m_296): 1 first_visits: 33
evaluation error (m_733): 26 first_visits: 26
evaluation error (m_593): 9 first_visits: 37
evaluation error (m_589): 4 first_visits: 38
evaluation error (m_318): 9 first_visits: 31
evaluation error (m_593): 1 first_visits: 40
evaluation error (m_1196): 0 first_visits: 34
evaluation error (m_318): 3 first_visits: 40
evaluation error (m_356): 0 first_visits: 38
evaluation error (m_356): 7 first_visits: 37
evaluation error (m_356): 9 first_visits: 39
evaluat